In [1]:
import warnings
warnings.simplefilter("ignore")

import pandas as pd
import numpy as np

import catboost as cb

from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

import datetime
from datetime import datetime, date

In [2]:
data = pd.read_csv('assignment_2_train.csv')

In [3]:
data.head(2)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.shape

(180000, 394)

**Задание 0:** выбрать любую модель машнного обучения и зафиксировать любой тип валидации. Обучить базовую модель и зафиксировать базовое качество модели. В каждом следующем задании нужно будет обучить выбранную модель и оценивать ее качество на зафиксированной схеме валидации. После каждого задания, требуется сделать вывод о достигаемом качестве модели, по сравнению с качестом из предыдущего шага.

In [5]:
target = data['isFraud']

data.drop(['isFraud', 'TransactionID'], axis=1, inplace=True)

In [6]:
categorical_features = data.select_dtypes(include=[np.object])
print(f'Всего категориальных признаков {categorical_features.shape[1]}')
# categorical_features = categorical_features.columns.tolist()

Всего категориальных признаков 14


In [7]:
for col in categorical_features:
    data[col] = data[col].astype(str)

In [8]:
x_train, x_valid, y_train, y_valid = train_test_split(
    data, target, train_size=0.7, shuffle=True, random_state=1,
)

In [9]:
train_pool = cb.Pool(x_train, y_train, cat_features=categorical_features)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=categorical_features)

In [10]:
model = cb.CatBoostClassifier(**cb_params)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)])

0:	test: 0.6645044	test1: 0.6640967	best: 0.6640967 (0)	total: 348ms	remaining: 5m 48s
100:	test: 0.8626380	test1: 0.8568925	best: 0.8568925 (100)	total: 25.7s	remaining: 3m 48s
200:	test: 0.8851982	test1: 0.8778457	best: 0.8778457 (200)	total: 51s	remaining: 3m 22s
300:	test: 0.8968462	test1: 0.8875002	best: 0.8875002 (300)	total: 1m 16s	remaining: 2m 58s
400:	test: 0.9003293	test1: 0.8903009	best: 0.8903009 (400)	total: 1m 41s	remaining: 2m 31s
500:	test: 0.9030912	test1: 0.8922876	best: 0.8922876 (500)	total: 2m 5s	remaining: 2m 5s
600:	test: 0.9062248	test1: 0.8946995	best: 0.8947030 (599)	total: 2m 30s	remaining: 1m 40s
700:	test: 0.9078557	test1: 0.8959960	best: 0.8960067 (666)	total: 2m 55s	remaining: 1m 14s
800:	test: 0.9079064	test1: 0.8960219	best: 0.8960221 (797)	total: 3m 21s	remaining: 49.9s
900:	test: 0.9081127	test1: 0.8962543	best: 0.8962553 (854)	total: 3m 45s	remaining: 24.8s
999:	test: 0.9086295	test1: 0.8967270	best: 0.8967292 (965)	total: 4m 10s	remaining: 0us

bes

**Задание 1:** признак TransactionDT - это смещение в секундах относительно базовой даты. Базовая дата - 2017-12-01, преобразовать признак TransactionDT в datetime, прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

In [11]:
base_date = int(datetime(2017, 12, 1).timestamp())
print(base_date)

1512075600


In [12]:
def change_date(df, feature):
    df[feature] = pd.to_datetime(df[feature] + base_date)
    df['Year'] = df[feature].dt.year
    df['Month'] = df[feature].dt.month
    df['Day_of_week'] = df[feature].dt.weekday
    df['Hour'] = df[feature].dt.hour
    df['Day'] = df[feature].dt.day
    df.drop([feature], axis=1, inplace=True)
    return df

In [13]:
x_train = change_date(x_train, "TransactionDT")
x_valid = change_date(x_valid, "TransactionDT")

In [14]:
train_pool = cb.Pool(x_train, y_train, cat_features=categorical_features)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=categorical_features)

In [15]:
cb_params = {
#     "n_estimators": 3000,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 100,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 100,
    "thread_count": 6,
    "random_seed": 42,
    "cat_features":categorical_features
}
model = cb.CatBoostClassifier(**cb_params)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)])

0:	test: 0.6229201	test1: 0.6242434	best: 0.6242434 (0)	total: 279ms	remaining: 4m 38s
100:	test: 0.8641673	test1: 0.8583656	best: 0.8583656 (100)	total: 26.6s	remaining: 3m 56s
200:	test: 0.8781339	test1: 0.8717821	best: 0.8717821 (200)	total: 52.5s	remaining: 3m 28s
300:	test: 0.8902962	test1: 0.8816370	best: 0.8816370 (300)	total: 1m 18s	remaining: 3m 2s
400:	test: 0.8938344	test1: 0.8845752	best: 0.8845752 (400)	total: 1m 44s	remaining: 2m 35s
500:	test: 0.8961948	test1: 0.8863814	best: 0.8863872 (497)	total: 2m 9s	remaining: 2m 8s
600:	test: 0.8985396	test1: 0.8877131	best: 0.8877131 (600)	total: 2m 34s	remaining: 1m 42s
700:	test: 0.9007850	test1: 0.8891580	best: 0.8891580 (700)	total: 2m 59s	remaining: 1m 16s
800:	test: 0.9008832	test1: 0.8892245	best: 0.8892254 (791)	total: 3m 25s	remaining: 51s
900:	test: 0.9009137	test1: 0.8892476	best: 0.8892492 (890)	total: 3m 49s	remaining: 25.3s
999:	test: 0.9009508	test1: 0.8892921	best: 0.8892922 (994)	total: 4m 14s	remaining: 0us

best

**Вывод** Новые признаки ухудшили модель на валидационной части

**Задание 2:** сделать конкатенацию признаков
* card1 + card2;
* card1 + card2 + card_3 + card_5;
* card1 + card2 + card_3 + card_5 + addr1 + addr2

Рассматривать их как категориальных признаки.

In [16]:
def feature_concatenation(df, features):
    new_name = f'{features[0]}'
    data = df[new_name].astype('str')
    for i in range(1, len(features)):
        new_name += f'_{features[i]}'
        df[features[i]] = df[features[i]].astype('str')
        data += df[features[i]]
    df[new_name] = data
    return df

In [17]:
x_train = feature_concatenation(x_train, ['card1', 'card2'])
x_valid = feature_concatenation(x_valid, ['card1', 'card2'])

x_train = feature_concatenation(x_train, ['card1', 'card2', 'card3', 'card5'])
x_valid = feature_concatenation(x_valid, ['card1', 'card2', 'card3', 'card5'])

x_train = feature_concatenation(x_train, ['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2'])
x_valid = feature_concatenation(x_valid, ['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2'])

In [22]:
x_train.head(2)

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,...,V338,V339,Year,Month,Day_of_week,Hour,Day,card1_card2,card1_card2_card3_card5,card1_card2_card3_card5_addr1_addr2
59549,117.0,W,16132,111.0,150.0,visa,226.0,debit,264.0,87.0,...,NaN,NaN,1970,1,3,0,1,16132111.0,16132111.0150.0226.0,16132111.0150.0226.0264.087.0
155490,20.0,H,5254,414.0,143.0,visa,226.0,debit,502.0,87.0,...,0.0,0.0,1970,1,3,0,1,5254414.0,5254414.0143.0226.0,5254414.0143.0226.0502.087.0


In [18]:
categorical_features = x_train.select_dtypes(include=[np.object])
print(f'Всего категориальных признаков {categorical_features.shape[1]}')
# categorical_features = categorical_features.columns.tolist()

Всего категориальных признаков 22


In [19]:
train_pool = cb.Pool(x_train, y_train, cat_features=categorical_features)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=categorical_features)

In [20]:
model = cb.CatBoostClassifier(**cb_params)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)])

0:	test: 0.5992861	test1: 0.5998528	best: 0.5998528 (0)	total: 183ms	remaining: 3m 3s
100:	test: 0.9565267	test1: 0.9103601	best: 0.9103627 (98)	total: 32.7s	remaining: 4m 50s
200:	test: 0.9642612	test1: 0.9194900	best: 0.9194900 (200)	total: 1m 6s	remaining: 4m 25s
300:	test: 0.9700471	test1: 0.9272188	best: 0.9272688 (299)	total: 1m 41s	remaining: 3m 55s
400:	test: 0.9739526	test1: 0.9313913	best: 0.9313917 (399)	total: 2m 15s	remaining: 3m 21s
500:	test: 0.9746192	test1: 0.9321232	best: 0.9321232 (497)	total: 2m 47s	remaining: 2m 46s
600:	test: 0.9747731	test1: 0.9323659	best: 0.9323738 (587)	total: 3m 19s	remaining: 2m 12s
700:	test: 0.9770346	test1: 0.9339696	best: 0.9339707 (695)	total: 3m 53s	remaining: 1m 39s
800:	test: 0.9773316	test1: 0.9344947	best: 0.9344995 (785)	total: 4m 25s	remaining: 1m 6s
900:	test: 0.9776985	test1: 0.9348502	best: 0.9348504 (898)	total: 4m 59s	remaining: 32.9s
999:	test: 0.9780721	test1: 0.9354535	best: 0.9354600 (997)	total: 5m 32s	remaining: 0us

b

**Вывод** новые признаки улучшили модель на 0,046 на валидации

**Задание 3:** Сделать FrequencyEncoder для признаков card1 - card6, addr1, addr2

In [23]:
def fe_features(df, features):
    for col in features:
        freq_encoder = df[col].value_counts(normalize=True)
        df[f'{col}_freq_enc'] = df[col].map(freq_encoder)
    return df

In [24]:
features = ['card1', 'card2', 'card3', 'card4', 'card5', 'card6']
x_train = fe_features(x_train, features)
x_valid = fe_features(x_valid, features)

In [26]:
x_train.head(2)

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,...,Day,card1_card2,card1_card2_card3_card5,card1_card2_card3_card5_addr1_addr2,card1_freq_enc,card2_freq_enc,card3_freq_enc,card4_freq_enc,card5_freq_enc,card6_freq_enc
59549,117.0,W,16132,111.0,150.0,visa,226.0,debit,264.0,87.0,...,1,16132111.0,16132111.0150.0226.0,16132111.0150.0226.0264.087.0,0.006421,0.075755,0.879872,0.657473,0.511449,0.681791
155490,20.0,H,5254,414.0,143.0,visa,226.0,debit,502.0,87.0,...,1,5254414.0,5254414.0143.0226.0,5254414.0143.0226.0502.087.0,0.000476,0.000595,0.001310,0.657473,0.511449,0.681791


In [27]:
train_pool = cb.Pool(x_train, y_train, cat_features=categorical_features)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=categorical_features)

In [28]:
model = cb.CatBoostClassifier(**cb_params)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)])

0:	test: 0.7167953	test1: 0.7194091	best: 0.7194091 (0)	total: 396ms	remaining: 6m 35s
100:	test: 0.9608374	test1: 0.9122512	best: 0.9122963 (99)	total: 34.1s	remaining: 5m 3s
200:	test: 0.9688743	test1: 0.9207989	best: 0.9207989 (200)	total: 1m 9s	remaining: 4m 34s
300:	test: 0.9769269	test1: 0.9301067	best: 0.9301067 (300)	total: 1m 44s	remaining: 4m 2s
400:	test: 0.9796746	test1: 0.9336514	best: 0.9336514 (400)	total: 2m 18s	remaining: 3m 27s
500:	test: 0.9800945	test1: 0.9342901	best: 0.9342901 (500)	total: 2m 51s	remaining: 2m 50s
600:	test: 0.9801531	test1: 0.9343397	best: 0.9343397 (600)	total: 3m 24s	remaining: 2m 15s
700:	test: 0.9803320	test1: 0.9345961	best: 0.9345961 (700)	total: 3m 57s	remaining: 1m 41s
800:	test: 0.9810986	test1: 0.9354569	best: 0.9354576 (799)	total: 4m 31s	remaining: 1m 7s
900:	test: 0.9825371	test1: 0.9373411	best: 0.9373453 (894)	total: 5m 6s	remaining: 33.6s
999:	test: 0.9827164	test1: 0.9374774	best: 0.9374891 (927)	total: 5m 38s	remaining: 0us

bes

**Вывод** значимого улучшения не наблюдается

**Задание 4:** Создать признаки на основе отношения: TransactionAmt к вычисленной статистике. Статистика - среднее значение / стандартное отклонение TransactionAmt, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

In [42]:
def relations(df, features, target_feature):
    for col in features:
        df[f'{col}_transform'] = df.groupby(col)[target_feature].transform('std')
    return df

In [44]:
target_feature = 'TransactionAmt'
features = ['card1', 'card2', 'card3', 'card4', 
            'card5', 'card6', 'addr1', 'addr2', 
            'card1_card2', 'card1_card2_card3_card5', 
            'card1_card2_card3_card5_addr1_addr2']

In [45]:
x_train = relations(x_train, features, target_feature)
x_valid = relations(x_valid, features, target_feature)

In [46]:
train_pool = cb.Pool(x_train, y_train, cat_features=categorical_features)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=categorical_features)

In [47]:
model = cb.CatBoostClassifier(**cb_params)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)])

0:	test: 0.6802052	test1: 0.6792717	best: 0.6792717 (0)	total: 396ms	remaining: 6m 35s
100:	test: 0.9574042	test1: 0.9071721	best: 0.9071735 (97)	total: 32.8s	remaining: 4m 52s
200:	test: 0.9631191	test1: 0.9150375	best: 0.9150667 (199)	total: 1m 6s	remaining: 4m 25s
300:	test: 0.9696687	test1: 0.9240224	best: 0.9240224 (300)	total: 1m 41s	remaining: 3m 55s
400:	test: 0.9747673	test1: 0.9298384	best: 0.9298384 (400)	total: 2m 15s	remaining: 3m 22s
500:	test: 0.9751133	test1: 0.9309039	best: 0.9309042 (498)	total: 2m 48s	remaining: 2m 47s
600:	test: 0.9751804	test1: 0.9310223	best: 0.9310223 (600)	total: 3m 20s	remaining: 2m 13s
700:	test: 0.9753503	test1: 0.9312671	best: 0.9312681 (696)	total: 3m 52s	remaining: 1m 39s
800:	test: 0.9793177	test1: 0.9343589	best: 0.9343589 (800)	total: 4m 27s	remaining: 1m 6s
900:	test: 0.9801956	test1: 0.9353096	best: 0.9353155 (875)	total: 4m 59s	remaining: 33s
999:	test: 0.9803514	test1: 0.9354236	best: 0.9354282 (966)	total: 5m 32s	remaining: 0us

be

**Вывод** значимого улучшения не наблюдается

**Задание 5:** Создать признаки на основе отношения: D15 к вычисленной статистике. Статистика - среднее значение / стандартное отклонение D15, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

In [48]:
target_feature = 'D15'

In [49]:
x_train = relations(x_train, features, target_feature)
x_valid = relations(x_valid, features, target_feature)

In [50]:
train_pool = cb.Pool(x_train, y_train, cat_features=categorical_features)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=categorical_features)

In [51]:
model = cb.CatBoostClassifier(**cb_params)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)])

0:	test: 0.6802052	test1: 0.6792717	best: 0.6792717 (0)	total: 393ms	remaining: 6m 32s
100:	test: 0.9557109	test1: 0.9091777	best: 0.9091777 (100)	total: 33.4s	remaining: 4m 57s
200:	test: 0.9704134	test1: 0.9229175	best: 0.9229175 (200)	total: 1m 8s	remaining: 4m 33s
300:	test: 0.9775864	test1: 0.9304557	best: 0.9304557 (300)	total: 1m 44s	remaining: 4m 2s
400:	test: 0.9797518	test1: 0.9332131	best: 0.9332131 (400)	total: 2m 18s	remaining: 3m 26s
500:	test: 0.9812381	test1: 0.9345431	best: 0.9345536 (498)	total: 2m 53s	remaining: 2m 52s
600:	test: 0.9824305	test1: 0.9353548	best: 0.9353548 (600)	total: 3m 27s	remaining: 2m 17s
700:	test: 0.9828159	test1: 0.9358760	best: 0.9358760 (700)	total: 4m	remaining: 1m 42s
800:	test: 0.9842216	test1: 0.9370225	best: 0.9370236 (798)	total: 4m 35s	remaining: 1m 8s
900:	test: 0.9843450	test1: 0.9371332	best: 0.9371332 (900)	total: 5m 9s	remaining: 34s
999:	test: 0.9845059	test1: 0.9373793	best: 0.9373795 (997)	total: 5m 41s	remaining: 0us

bestTes

**Вывод** значимого улучшения не наблюдается

**Задание 6:** выделить дробную часть и целую часть признака TransactionAmt в два отдельных признака. После создать отдельных признак - логарифм от TransactionAmt

In [53]:
def transform_feature(df, target_feature):
    df[f'{target_feature}_integer'] = df[target_feature].astype(int)
    df[f'{target_feature}_fractional'] = df[target_feature] - df[f'{target_feature}_integer']
    df[f'{target_feature}_log'] = np.log(df[target_feature])
    
    return df

In [54]:
x_train = transform_feature(x_train, 'TransactionAmt')
x_valid = transform_feature(x_valid, 'TransactionAmt')

In [55]:
train_pool = cb.Pool(x_train, y_train, cat_features=categorical_features)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=categorical_features)

In [56]:
model = cb.CatBoostClassifier(**cb_params)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)])

0:	test: 0.6458487	test1: 0.6431004	best: 0.6431004 (0)	total: 403ms	remaining: 6m 42s
100:	test: 0.9573587	test1: 0.9089664	best: 0.9089664 (100)	total: 34.2s	remaining: 5m 3s
200:	test: 0.9730168	test1: 0.9230775	best: 0.9230775 (200)	total: 1m 10s	remaining: 4m 39s
300:	test: 0.9769820	test1: 0.9303192	best: 0.9303192 (300)	total: 1m 44s	remaining: 4m 3s
400:	test: 0.9788528	test1: 0.9335611	best: 0.9335611 (400)	total: 2m 19s	remaining: 3m 28s
500:	test: 0.9792792	test1: 0.9339926	best: 0.9339926 (500)	total: 2m 52s	remaining: 2m 51s
600:	test: 0.9793842	test1: 0.9343533	best: 0.9343533 (600)	total: 3m 25s	remaining: 2m 16s
700:	test: 0.9807009	test1: 0.9361724	best: 0.9361724 (700)	total: 3m 59s	remaining: 1m 42s
800:	test: 0.9817468	test1: 0.9374610	best: 0.9374619 (797)	total: 4m 34s	remaining: 1m 8s
900:	test: 0.9819311	test1: 0.9376998	best: 0.9377000 (893)	total: 5m 7s	remaining: 33.7s
999:	test: 0.9822180	test1: 0.9379532	best: 0.9379533 (998)	total: 5m 40s	remaining: 0us

b

**Вывод** значимого улучшения не наблюдается

**Вывод** Максимальный прирост качества был на 2 шаге (конкатенация признаков). Все последующие шаги улучшили базовую модель на 0,0025 на валидации.

In [58]:
x_train.head()

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,...,card5_transform,card6_transform,addr1_transform,addr2_transform,card1_card2_transform,card1_card2_card3_card5_transform,card1_card2_card3_card5_addr1_addr2_transform,TransactionAmt_integer,TransactionAmt_fractional,TransactionAmt_log
59549,117.00,W,16132,111.0,150.0,visa,226.0,debit,264.0,87.0,...,187.926834,183.157453,187.013034,186.132957,176.626144,176.626144,109.918606,117,0.00,4.762174
155490,20.00,H,5254,414.0,143.0,visa,226.0,debit,502.0,87.0,...,187.926834,183.157453,191.040339,186.132957,205.028961,205.028961,203.236703,20,0.00,2.995732
32608,57.95,W,6692,555.0,150.0,visa,226.0,debit,485.0,87.0,...,187.926834,183.157453,178.039534,186.132957,95.247047,95.247047,112.697826,57,0.95,4.059581
21174,107.95,W,4806,490.0,150.0,visa,226.0,debit,315.0,87.0,...,187.926834,183.157453,192.640387,186.132957,201.568789,201.568789,200.606969,107,0.95,4.681668
87435,150.00,R,3652,399.0,150.0,american express,150.0,credit,204.0,87.0,...,216.622691,171.185662,187.899064,186.132957,NaN,NaN,NaN,150,0.00,5.010635


In [59]:
x_train.shape

(125999, 419)